1. find the correct expression of the normalization constant in Eq. (8)

In [2]:
import numpy as np
import scipy
import sys
import scipy.special
import scipy.constants as const
from math import factorial
from scipy.special import hermite
import matplotlib.pyplot as plt

In [3]:
#constants

v_0R = 200 #Mev
v_0t = 178 #Mev
v_0s = 91.85 #Mev

k_R = 1.487 #fm^-2 should we change in A?
k_t = 0.639 #fm^-2
k_s = 0.465 #fm^-2

m_p = 938.272 # mass of proton in MeV/c^2
m_n = 939.565 # mass of neutron in MeV/c^2
m_e = 0.511 # mass of electron in MeV/c^2
mu = m_p*m_n/(m_p+m_n) #reduced mass in amu

h_bar_Js = const.hbar #hbar in Js
h_bar = h_bar_Js / (1.6e-19) # hbar in eVs
c=const.c

a = 1e-3 * h_bar**2 *(c*1e10)**2/(2*mu) #1e-3 for conversion meV, eV -> meV - 1e-10 for conversion m -> Å
print(a) #eV*Angstrom^2, 


4.158398182328096


normalisation:
\begin{equation}
\tilde{N}_n  = \left[\frac{\sqrt{\nu}}{2\pi^{3/2}2^{2n+1} (2n+1)!}\right]^{1/2}
\end{equation}

In [4]:
#FUNCTIONS
def norm_constant(n, nu):
    '''
    this function returns the normalisation constant for the n-th order radial wavefunction 
    '''
    denom = 4*np.pi**(3/2) * 2**(2*n+1) / (2*nu)**0.5 * factorial(2*n+1)
    return (2 / denom)**(1/2) #the factor sqrt 2 comes from hermites being even, and our given normalisation being between -inf and +inf

def R_n(n,r,nu):
    '''
    this function returns the n-th order radial wf of eigenstates of 3d HO
    '''
    return norm_constant(n,nu)* np.exp(-nu*r**2)* hermite(2*n+1)((2*nu)**0.5*r) / r



Check normalisation quickly with numpy.trapz. One is free to change n1 and n2 to see if the radial wf's are orthonormal

In [6]:
n1 = 5
n2 = 5
nu = 1

r = np.linspace(0.1, 10, 1000)

R1 = [R_n(n1, r, nu) for r in r]
R2 = [R_n(n2, r, nu) for r in r]

integrand = [4*np.pi*r1 * r2 * r**2 for r1, r2, r in zip(R1, R2, r)]

norm = np.trapz(integrand, r)
print("The scalar product between the",n1, "wavefunction and the", n2, "wavefunction is",norm)

The scalar product between the 5 wavefunction and the 5 wavefunction is 0.9947270635579459


Working on spin-isospin

In [11]:
#DEFINING THE STATES
up = 1
down = -1
prefactor = 1 / np.sqrt(2)
# A = (|↑↑↑↓⟩ − |↑↓↑↑⟩)
A = [ ((up, up, up, down), prefactor), ((up, down, up, up), -prefactor) ]

# B = (|↑↑↓↓⟩ − |↓↓↑↑⟩)
B = [ ((up, up, down, down), prefactor), ((down, down, up, up), -prefactor) ]

# C = (|↓↑↓↓⟩ − |↓↓↓↑⟩)
C = [ ((down, up, down, down), prefactor), ((down, down, down, up), -prefactor) ]

# D = (|↓↑↑↓⟩ − |↑↓↓↑⟩)
D = [ ((down, up, up, down), prefactor), ((up, down, down, up), -prefactor)] 


[((1, 1, 1, -1), 0.7071067811865475), ((1, -1, 1, 1), -0.7071067811865475)]


calculate kinetic energy matrix element
\begin{equation}
\hat{K} =  -\frac{\hbar^2}{2\mu} \nabla ^2
\end{equation}
with 
\begin{equation}
\nabla^2 R_n = \frac{1}{r} \frac{\partial ^2}{\partial r ^2} (r R_n) + \text{angular}
\end{equation}
exploiting orthornmality of the Hermites, we get
\begin{equation}
\hat{K} R_n =- \frac{\hbar^2\nu}{\mu r} \tilde{N}_n(\nu) e^{-x^2} \left[2 (-1+2x^2) H_{2n+1} - 8 (2n+1) x H_{2n} + 2(2n+1) H_{2n-1}\right]
\end{equation}
and 
\begin{equation}
\bra{R_m} \hat{K} \ket{R_n} = 
\end{equation}

In this section we try to evaluate the matrix elements of the operator
$$
\hat{V}=\frac{1}{2}(1+\hat{P}^{\sigma})V_t(r)+\frac{1}{2}(1-\hat{P}^{\sigma})V_s(r)
$$


In [10]:
# Compute inner product between two spin states

def inner_product(state1, state2):
    total = 0
    for (s1, c1) in state1: #represent full basis states: s1 = (down, up, down, down) for example, c1 is prefactor
        for (s2, c2) in state2:
                S1 = tuple(x for x in s1)  # multiply the coefficients with the spin states
                S2 = tuple(x for x in s2)  # multiply the coefficients with the spin states
                if S1 == S2:
                    total += c1 * c2
                    #print(total)
    return total


#check if it works
print("Inner product is",inner_product(B,B))
print("Inner product is",inner_product(A,A))
print("Inner product is",inner_product(C,C))
print("Inner product is",inner_product(D,D))
print("Inner product is",inner_product(A,B))
print("Inner product is",inner_product(A,C))
print("Inner product is",inner_product(A,D))
print("Inner product is",inner_product(B,C))
print("Inner product is",inner_product(B,D))
print("Inner product is",inner_product(C,D))

def spin_exchange(state):
    exchanged_state = []
    for spin, coeff in state:
        a, b, c, d = spin  # Unpack the original spin tuple (safe and clean)
        new_spin = (c, b, a, d)  # Swap the first three elements
        exchanged_state.append((new_spin, coeff))
    return exchanged_state

#H = spin_exchange(B)
#print("\n",H,"\n",B)

def V_R(r):
    return v_0R * np.exp(-k_R*r**2) # MeV

def V_t(r):
    return v_0t * np.exp(-k_t*r**2) # MeV

def V_s(r):
    return v_0s * np.exp(-k_s*r**2) # MeV


def V_matrix_elements(state1, state2, r):
    '''
    This function returns the potential energy between two states at a distance r.
    '''
    # Apply spin exchange to state2
    exchanged_state2 = spin_exchange(state2)
    
    # Compute the two potential terms (you may need to adjust this depending on the context)
    V_t_value = V_t(r)
    V_s_value = V_s(r)
    
    Vij =  inner_product(state1,state2)
    Vij_P = inner_product(state1, exchanged_state2)

    return (0.5 * (Vij + Vij_P) * V_t_value + 0.5 * (Vij - Vij_P) * V_s_value) 

#r = 1.0  # radial distance in fm
#print("The potential energy between A and B is", V_matrix_elements(B,D,r))

Inner product is 0.9999999999999998
Inner product is 0.9999999999999998
Inner product is 0.9999999999999998
Inner product is 0.9999999999999998
Inner product is 0
Inner product is 0
Inner product is 0
Inner product is 0
Inner product is 0
Inner product is 0


In [44]:
states = [A, B, C, D]
labels = ["A", "B", "C", "D"]

r = 1.0  # radial distance in fm

# Compute the full matrix
matrix = np.zeros((4, 4))

for i, state_i in enumerate(states):
    for j, state_j in enumerate(states):
        matrix[i, j] = V_matrix_elements(state_i, state_j, r)

# Display results nicely
print("Matrix elements ⟨X|V(r)|Y⟩ with r =", r, "fm\n")
for i in range(4):
    for j in range(4):
        print(f"⟨{labels[i]}|V|{labels[j]}⟩ = {matrix[i,j]: .4f} MeV")
    print()

Matrix elements ⟨X|V(r)|Y⟩ with r = 1.0 fm

⟨A|V|A⟩ =  93.9520 MeV
⟨A|V|B⟩ =  0.0000 MeV
⟨A|V|C⟩ =  0.0000 MeV
⟨A|V|D⟩ =  0.0000 MeV

⟨B|V|A⟩ =  0.0000 MeV
⟨B|V|B⟩ =  75.8231 MeV
⟨B|V|C⟩ =  0.0000 MeV
⟨B|V|D⟩ =  18.1289 MeV

⟨C|V|A⟩ =  0.0000 MeV
⟨C|V|B⟩ =  0.0000 MeV
⟨C|V|C⟩ =  93.9520 MeV
⟨C|V|D⟩ =  0.0000 MeV

⟨D|V|A⟩ =  0.0000 MeV
⟨D|V|B⟩ =  18.1289 MeV
⟨D|V|C⟩ =  0.0000 MeV
⟨D|V|D⟩ =  75.8231 MeV



Now we focus on the radial component.
We express the kinetic energy term in spherical coordinates 
$$
\hat{H}=-\frac{\hbar^2}{2\mu}\nabla^2 = -\frac{\hbar^2}{2\mu} \left[ \frac{1}{r} \frac{\partial}{\partial r^2} r + \frac{1}{r^2 \sin \theta} \frac{\partial}{\partial \theta} \left( \sin \theta \frac{\partial}{\partial \theta} \right) + \frac{1}{r^2 \sin^2 \theta} \frac{\partial^2}{\partial \phi^2} \right]
$$
so that we can neglect the centrifugal terms because we are considering only states in an s-wave state $(l=0)$, which yields to $$\hat{L}f(r,\theta,\phi)=\hbar^2 l(l+1) f(r,\theta,\phi)=0$$
To get the matrix elements regarading the kinetic energy we have to solve
$$
-\frac{\hbar^2}{2\mu} \frac{1}{r} \frac{\partial}{\partial r^2} r  R_n (r)
$$
where $$R_n(r)=N_n(\nu)\frac{exp(-\nu r^2)}{r} H_{2n+1}(\sqrt{2\pi}r)$$.
Then we can add the diagonal term carried by $$V_R(r)$$



In [ ]:
dim = 10
H = np.zeros[dim,dim]

for i in range(dim):
    for j in range(dim):
        if i == j:
            V_R_value = V_R(r)
            N = norm_constant(dim, nu)^2
            H[i,j] = N^2*8*(2*np.pi)**2*nu*r*dim + V_R_value 
        